In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay
df = pd.read_csv('heart.csv')
print(df.shape)
print(df.columns)
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())
df.fillna(df.median(numeric_only=True), inplace=True)
sns.heatmap(df.corr(), annot=True)
plt.title('Correlation matrix')
plt.show()
X = df.drop(columns=['target'])
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
lr = LogisticRegression(max_iter=1000)
dt = DecisionTreeClassifier(random_state=42)
lr.fit(X_train_scaled, y_train)
dt.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test_scaled)
y_pred_dt = dt.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)
acc_dt = accuracy_score(y_test, y_pred_dt)
proba_lr = lr.predict_proba(X_test_scaled)[:,1]
proba_dt = dt.predict_proba(X_test)[:,1]
auc_lr = roc_auc_score(y_test, proba_lr)
auc_dt = roc_auc_score(y_test, proba_dt)
print(f'Logistic Regression Accuracy: {acc_lr:.4f}, ROC-AUC: {auc_lr:.4f}')
print(f'Decision Tree Accuracy: {acc_dt:.4f}, ROC-AUC: {auc_dt:.4f}')
fpr_lr, tpr_lr, _ = roc_curve(y_test, proba_lr)
fpr_dt, tpr_dt, _ = roc_curve(y_test, proba_dt)
plt.plot(fpr_lr, tpr_lr, label=f'LogReg AUC={auc_lr:.3f}')
plt.plot(fpr_dt, tpr_dt, label=f'DecTree AUC={auc_dt:.3f}')
plt.plot([0,1],[0,1],'--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.show()
cm = confusion_matrix(y_test, y_pred_lr)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion Matrix - Logistic Regression')
plt.show()
feature_importances_dt = pd.Series(dt.feature_importances_, index=X.columns).sort_values(ascending=False)
print('Decision Tree Feature Importances:')
print(feature_importances_dt.head(10))
coef_lr = pd.Series(lr.coef_[0], index=X.columns).sort_values(key=abs, ascending=False)
print('Logistic Regression Coefficients (by absolute impact):')
print(coef_lr.head(10))
